# Libs

In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, balanced_accuracy_score, roc_curve, roc_auc_score
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
import plotly.graph_objects as go
from utils.futurai_ppd import drop_transitorio_desligado
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelBinarizer
from sklearn.linear_model import RidgeClassifierCV
from sktime.transformations.panel.rocket import Rocket
from collections import Counter
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh import select_features
from scipy.stats import kurtosis
from sktime.datatypes._panel._convert import from_long_to_nested

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, LSTM, Dense, Dropout, GlobalMaxPooling1D, LayerNormalization, MultiHeadAttention, Add, Flatten, Concatenate

import warnings
warnings.filterwarnings('ignore')

# Import dataset

In [ ]:
base_name = 'Depurador 762-28-006 - Cozimento'
timestamp = "Timestamp"

df_dataset = pd.read_csv('data/' + base_name + '.csv', sep=";", decimal=".", encoding="utf-8-sig")
df_dataset[timestamp] = pd.to_datetime(df_dataset[timestamp], format="%Y-%m-%d %H:%M:%S")

## Drop columns with NaN values, constant values or irrelevant to the analysis
df_dataset.drop(columns=["762H0336.PV", "762H0342.PV", "762N0015.SP", "762P0013.SP", "762-34-073.CR", "762N0015.OP"], inplace=True, errors='ignore')

df_dataset.dropna(inplace=True)

print(f"Dataset shape: {df_dataset.shape}")

list_variables = df_dataset.columns.tolist()
df_dataset.head()

## Remove periods Off

In [ ]:
pre_process = []
pp_var_ref_desligado = "762-28-006.CR"
pp_valor_ref_desligado = 5
pp_tempo_ref_desligado = 0
pp_pre_corte_transitorio = 0
pp_pos_corte_transitorio = 0
pre_process.append(  
{
   "after_cut": pp_pos_corte_transitorio,
   "interval_off": pp_tempo_ref_desligado,
   "limit_off": pp_valor_ref_desligado,
   "pre_cut": pp_pre_corte_transitorio,
   "variable_off": pp_var_ref_desligado
  })

for pro in pre_process:
    df_dataset,_,_ = drop_transitorio_desligado(df_dataset,pro["variable_off"],pro["limit_off"],pro["interval_off"],timestamp,pre_corte=pro["pre_cut"],pos_corte=pro["after_cut"])
print(f"Dataset shape: {df_dataset.shape}")
df_dataset.head()

## Create label for anomaly

In [ ]:
periodos_de_Falhas = [
    (pd.Timestamp('2024-05-03 11:00:00'), pd.Timestamp('2024-05-03 11:35:00')),
    (pd.Timestamp('2024-06-25 17:20:00'), pd.Timestamp('2024-08-02 14:00:00')),
    (pd.Timestamp('2024-10-19 10:40:00'), pd.Timestamp('2024-10-19 10:50:00')),
    (pd.Timestamp('2024-10-19 10:40:00'), pd.Timestamp('2024-10-19 10:50:00')),
    (pd.Timestamp('2024-10-21 12:00:00'), pd.Timestamp('2024-10-22 00:35:00')),
    (pd.Timestamp('2024-10-24 03:10:00'), pd.Timestamp('2024-10-27 00:00:00')),
    (pd.Timestamp('2024-11-14 06:40:00'), pd.Timestamp('2024-11-14 19:45:00')),
    (pd.Timestamp('2024-11-25 21:45:00'), pd.Timestamp('2024-11-25 22:03:00')),
    (pd.Timestamp('2024-11-27 15:00:00'), pd.Timestamp('2024-11-27 15:07:00')),
    (pd.Timestamp('2024-11-27 16:04:00'), pd.Timestamp('2024-11-26 16:11:00')),
    (pd.Timestamp('2024-11-30 13:30:00'), pd.Timestamp('2024-11-30 15:52:00')),
    (pd.Timestamp('2024-12-09 20:30:00'), pd.Timestamp('2024-12-11 07:00:00')),
    (pd.Timestamp('2025-03-05 15:17:00'), pd.Timestamp('2025-03-05 15:24:00')),
    (pd.Timestamp('2025-03-15 18:30:00'), pd.Timestamp('2025-03-15 19:15:00')),
    (pd.Timestamp('2025-03-18 11:40:00'), pd.Timestamp('2025-03-18 20:00:00')),
    (pd.Timestamp('2025-06-16 15:20:00'), pd.Timestamp('2025-06-17 07:38:00')),
]

df_dataset['Falhas'] = 0
for inicio, fim in periodos_de_Falhas:
    df_dataset.loc[(df_dataset[timestamp] >= inicio) & (df_dataset[timestamp] <= fim), 'Falhas'] = 1
df_dataset.head()

## Import TAGs and descriptions

In [ ]:
df_subsistema = pd.read_csv('data/'+ base_name + '_subsistema.csv', sep=";", decimal=".", encoding="utf-8-sig")
df_subsistema

## Plot variables

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_dataset['Timestamp'],
    y=df_dataset["762P0034.PV"],
    mode='lines',
    name='762P0034.PV',
    line=dict(color='black')
))

fig.add_trace(go.Scatter(
    x=df_dataset['Timestamp'],
    y=df_dataset["Falhas"],
    mode='lines',
    name='Falhas',
    line=dict(color='red')
))

fig.update_layout(
    template='plotly_white',
    hovermode='x unified'
)
fig.show()

## Feature extraction

### Extract Features with tsfresh

In [ ]:
df = df_dataset.set_index(timestamp)

variables = df.drop(columns=["Falhas"])
labels = df["Falhas"]

# parâmetros de janela
window = pd.Timedelta("60min")
step = pd.Timedelta("30min")

# criar índices baseados em passo fixo
df = df.copy()
df["window_start"] = df.index.floor(step)
df["window_id"] = ((df.index - df["window_start"]) // step).cumsum()

# corrigir para garantir tamanho de janela correto
valid_windows = df.groupby("window_id").filter(
    lambda x: (x.index.max() - x.index.min()) >= window
)

# long format direto
long_format = (
    valid_windows
    .reset_index()
    .melt(id_vars=[timestamp, "window_id"], var_name="variable", value_name="value")
)

# extrair features
X_features = extract_features(
    long_format,
    column_id="window_id",
    column_sort=timestamp,
    column_kind="variable",
    column_value="value",
    n_jobs=1
)

X_features = impute(X_features)

# rótulo: 1 se houver falha dentro da janela
y_features = labels.groupby(df["window_id"]).apply(lambda x: int(x.any()))
y_features = y_features.loc[X_features.index]

print("Shape final de X_features:", X_features.shape)
print("Shape final de y_features:", y_features.shape)

### Extract Features with functions

In [ ]:
## utils
def shannon_entropy(arr, bins=10):
    if arr.size == 0:
        return np.nan
    p, _ = np.histogram(arr, bins=bins, density=True)
    p = p[p > 0]
    if p.size == 0:
        return np.nan
    return -np.sum(p * np.log(p))

def safe_autocorr(s, lag=1):
    try:
        return s.autocorr(lag=lag)
    except Exception:
        return np.nan

# function to extract features from a window
def features_from_window(g):
    out = {}
    for col in g.columns:
        s = g[col].dropna()
        prefix = col
        if s.empty:
            for name in ["mean","std","var","min","max","median","skew","kurtosis",
                         "q25","q75","iqr","amplitude","energy","entropy",
                         "autocorr_lag1","autocorr_lag2","coeff_var"]:
                out[f"{prefix}_{name}"] = np.nan
            continue

        arr = s.values
        out[f"{prefix}_mean"] = s.mean()
        out[f"{prefix}_std"] = s.std()
        out[f"{prefix}_var"] = s.var()
        out[f"{prefix}_min"] = s.min()
        out[f"{prefix}_max"] = s.max()
        out[f"{prefix}_median"] = s.median()
        out[f"{prefix}_skew"] = s.skew()
        out[f"{prefix}_kurtosis"] = float(kurtosis(arr, fisher=True))
        out[f"{prefix}_q25"] = np.percentile(arr, 25)
        out[f"{prefix}_q75"] = np.percentile(arr, 75)
        out[f"{prefix}_iqr"] = out[f"{prefix}_q75"] - out[f"{prefix}_q25"]
        out[f"{prefix}_amplitude"] = out[f"{prefix}_max"] - out[f"{prefix}_min"]
        out[f"{prefix}_energy"] = np.sum(arr ** 2)
        out[f"{prefix}_entropy"] = shannon_entropy(arr, bins=10)
        out[f"{prefix}_autocorr_lag1"] = safe_autocorr(s, lag=1)
        out[f"{prefix}_autocorr_lag2"] = safe_autocorr(s, lag=2)
        mean_val = out[f"{prefix}_mean"]
        out[f"{prefix}_coeff_var"] = (out[f"{prefix}_std"] / mean_val) if mean_val not in (0, np.nan) else np.nan

    return pd.Series(out)


# params
window = "60min"
step = "30min"

df = df_dataset.set_index(timestamp)
variables = df.drop(columns=["Falhas"])
labels = df["Falhas"]

# build sliding windows
starts = pd.date_range(df.index.min(), df.index.max() - pd.Timedelta(window), freq=step)

X_list, y_list, idx_list = [], [], []

for start in starts:
    end = start + pd.Timedelta(window)
    window_vars = variables.loc[start:end]
    window_labels = labels.loc[start:end]

    if window_vars.empty or window_labels.empty:
        continue

    # extract features
    feats = features_from_window(window_vars)
    X_list.append(feats)

    # if any failure in the window, label as 1
    y_list.append(int(window_labels.any()))

    # index representing the center of the window
    idx_list.append(start + (pd.Timedelta(window) / 2))

# build final DataFrames
X_features = pd.DataFrame(X_list, index=idx_list)
y_features = pd.Series(y_list, index=idx_list, name="Falhas")

# remove samples with any NaN in X_features
mask_valid = ~X_features.isna().any(axis=1)
X_features = X_features.loc[mask_valid]
y_features = y_features.loc[mask_valid]

print("Shape final de X_features:", X_features.shape)
print("Shape final de y_features:", y_features.shape)

### Basic Feature Extraction

In [ ]:
df = df_dataset.set_index(timestamp)

variables = df.drop(columns=["Falhas"])
labels = df["Falhas"]

# windows parameters
window = pd.Timedelta("60min")
step = pd.Timedelta("30min")

# init point of each window
starts = pd.date_range(df.index.min(), df.index.max() - window, freq=step)

X_list, y_list, idx_list = [], [], []

for start in starts:
    end = start + window
    window_vars = variables.loc[start:end]
    window_labels = labels.loc[start:end]

    if window_vars.empty or window_labels.empty:
        continue

    # Extract basic features
    feats = window_vars.agg(["mean", "std", "min", "max", "median", "skew", "var"]).T
    feats.index = [f"{col}" for col in feats.index]  # keep variable names
    feats = feats.stack()  # "long"
    feats.index = [f"{var}_{stat}" for var, stat in feats.index]
    X_list.append(feats)

    # any failure in the window = label 1
    y_list.append(int(window_labels.any()))

    # index represent the center of the window
    idx_list.append(start + window / 2)

# Final DataFrames
X_features = pd.DataFrame(X_list, index=idx_list)
y_features = pd.Series(y_list, index=idx_list, name="Falhas")

# Clean NaN values
X_features = X_features.dropna()
y_features = y_features.loc[X_features.index]

print("Shape final de X_features:", X_features.shape)
print("Shape final de y_features:", y_features.shape)

### Train Test Split

In [ ]:
X_features_train, X_features_test, y_features_train, y_features_test = train_test_split(
    X_features, 
    y_features, 
    test_size=0.3, 
    random_state=42, 
    shuffle=False
)

## Scalling training set
scaler = StandardScaler()
scaler.fit(X_features_train)
X_features_train = pd.DataFrame(scaler.transform(X_features_train), columns=X_features_train.columns, index=X_features_train.index)
X_features_test = pd.DataFrame(scaler.transform(X_features_test), columns=X_features_test.columns, index=X_features_test.index)


## Verify class distribution in train and test sets
print("Class distribution in training set (%):")
print(y_features_train.value_counts(normalize=True) * 100)
print("\nClass distribution in test set (%):")
print(y_features_test.value_counts(normalize=True) * 100)

### SMOTE training set

In [ ]:
sm = SMOTE(random_state=42)
X_features_train_balanced, y_features_train_balanced = sm.fit_resample(X_features_train, y_features_train)

## Verify class distribution in train and test sets
print("Class distribution in training set (%):")
print(y_features_train_balanced.value_counts(normalize=True) * 100)

In [ ]:
# X_features_train_balanced = X_features_train
# y_features_train_balanced = y_features_train

# Random Forest

In [ ]:
rf_clf = RandomForestClassifier(
    n_estimators=400, 
    criterion='gini', 
    max_depth=None, 
    min_samples_split=2, 
    min_samples_leaf=1, 
    min_weight_fraction_leaf=0.0, 
    max_features='sqrt', 
    max_leaf_nodes=None, 
    min_impurity_decrease=0.0, 
    bootstrap=True, 
    oob_score=False, 
    n_jobs=None, 
    random_state=42, 
    verbose=0, 
    warm_start=False, 
    class_weight=None, 
    ccp_alpha=0.0, 
    max_samples=None, 
    monotonic_cst=None
)

## Cross validation
k_folds = KFold(n_splits = 5)
scores = cross_val_score(rf_clf, X_features_train_balanced, y_features_train_balanced, cv = k_folds)
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

## Fit model
rf_clf.fit(X_features_train_balanced, y_features_train_balanced)

## Predict and evaluate
y_pred = rf_clf.predict(X_features_test)

accuracy = balanced_accuracy_score(y_features_test, y_pred)
f1 = f1_score(y_features_test, y_pred, average='weighted')
recall = recall_score(y_features_test, y_pred, average='weighted')
precision = precision_score(y_features_test, y_pred, average='weighted')
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")

## Plot ROC Curve
y_proba = rf_clf.predict_proba(X_features_test)

lb = LabelBinarizer()
y_test_bin = lb.fit_transform(y_features_test)

y_test_bin = y_test_bin.ravel()
y_scores = y_proba[:, 1]
fpr, tpr, _ = roc_curve(y_test_bin, y_scores)
roc_auc = roc_auc_score(y_test_bin, y_scores)

plt.figure(figsize=(6, 6))
plt.plot(fpr, tpr, color="blue", lw=2, label=f"ROC curve (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], color="gray", lw=1, linestyle="--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve - RandomForest")
plt.legend(loc="lower right")
plt.grid(alpha=0.3)
plt.show()

In [ ]:
y_pred_rf = rf_clf.predict(X_features_test.to_numpy())

y_features_test.sort_index(inplace=True)
X_features_test.sort_index(inplace=True)

fig_pred_rf = go.Figure()
fig_pred_rf.add_trace(go.Scatter(
    x = X_features_test.index,
    y=y_pred_rf,
    name='Predict',
    mode="lines",
    line=dict(color='black')
))
fig_pred_rf.add_trace(go.Scatter(
    y=y_features_test,
    x = X_features_test.index,
    name='Real',
    mode="lines",
    line=dict(color='red')
))
fig_pred_rf.update_layout(
    template='plotly_white',
    hovermode='x unified',
    title="RandomForest Predictions vs Real Values"
)
fig_pred_rf.show()

In [ ]:
import shap

explainer = shap.TreeExplainer(rf_clf)
# Shap values of the test set
shap_values = explainer.shap_values(X_features_test.to_numpy())

# Summary plot mean feature importance
shap.summary_plot(shap_values, X_features_test, plot_type="bar")
# 2. Beeswarm plot impact of each variable on each prediction
shap.summary_plot(shap_values, X_features_test)

# XGBoost

In [ ]:
xgb_clf = XGBClassifier(
    n_estimators=400,      # number of trees
    learning_rate=0.1,     # learning rate (smaller is more stable, but needs more trees)
    max_depth=6,           # maximum tree depth
    subsample=0.8,         # fraction of samples used in each tree
    colsample_bytree=0.8,  # fraction of features used in each tree
    random_state=42,
    use_label_encoder=False,
    eval_metric="logloss"
)

## Cross validation
k_folds = KFold(n_splits = 5)
scores = cross_val_score(xgb_clf, X_features_train_balanced.to_numpy(), y_features_train_balanced.to_numpy(), cv = k_folds)
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

## Fit model
xgb_clf.fit(X_features_train_balanced.to_numpy(), y_features_train_balanced.to_numpy())

## Predict and evaluate
y_pred = xgb_clf.predict(X_features_test.to_numpy())

accuracy = balanced_accuracy_score(y_features_test, y_pred)
f1 = f1_score(y_features_test, y_pred, average='weighted')
recall = recall_score(y_features_test, y_pred, average='weighted')
precision = precision_score(y_features_test, y_pred, average='weighted')
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")

## Plot ROC Curve
y_proba = xgb_clf.predict_proba(X_features_test.to_numpy())

lb = LabelBinarizer()
y_test_bin = lb.fit_transform(y_features_test.to_numpy())

y_test_bin = y_test_bin.ravel()
y_scores = y_proba[:, 1]
fpr, tpr, _ = roc_curve(y_test_bin, y_scores)
roc_auc = roc_auc_score(y_test_bin, y_scores)

plt.figure(figsize=(6, 6))
plt.plot(fpr, tpr, color="blue", lw=2, label=f"ROC curve (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], color="gray", lw=1, linestyle="--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve - RandomForest")
plt.legend(loc="lower right")
plt.grid(alpha=0.3)
plt.show()

In [ ]:
y_pred_xgb = xgb_clf.predict(X_features_test.to_numpy())

y_features_test.sort_index(inplace=True)

fig_pred_xgb = go.Figure()
fig_pred_xgb.add_trace(go.Scatter(
    x = X_features_test.index,
    y=y_pred_xgb,
    name='Predict',
    mode="lines",
    line=dict(color='black')
))
fig_pred_xgb.add_trace(go.Scatter(
    y=y_features_test,
    x = X_features_test.index,
    name='Real',
    mode="lines",
    line=dict(color='red')
))
fig_pred_xgb.update_layout(
    template='plotly_white',
    hovermode='x unified',
    title="XGBoost Predictions vs Real Values"
)
fig_pred_xgb.show()

In [ ]:
import shap

explainer = shap.TreeExplainer(xgb_clf)
# Shap values of the test set
shap_values = explainer.shap_values(X_features_test.to_numpy())

# Summary plot mean feature importance
shap.summary_plot(shap_values, X_features_test, plot_type="bar")
# 2. Beeswarm plot impact of each variable on each prediction
shap.summary_plot(shap_values, X_features_test)

# New Data Classification

## Import and pre-process Data

In [ ]:
base_name = 'Depurador 762-28-006'
timestamp = "Timestamp"

df_newdata = pd.read_csv('data/' + base_name + '_teste.csv', sep=";", decimal=".", encoding="utf-8-sig")
df_newdata = df_newdata[list_variables]

df_newdata[timestamp] = pd.to_datetime(df_newdata[timestamp], format="%Y-%m-%d %H:%M:%S")
df_newdata.drop_duplicates(subset=[timestamp], keep='first', inplace=True)
df_newdata.sort_values(by=timestamp, inplace=True)
df_newdata.dropna(inplace=True)
print(f"Dataset shape: {df_newdata.shape}")


pre_process = []
pp_var_ref_desligado = "762-28-006.CR"
pp_valor_ref_desligado = 5
pp_tempo_ref_desligado = 0
pp_pre_corte_transitorio = 0
pp_pos_corte_transitorio = 0
pre_process.append(  
{
   "after_cut": pp_pos_corte_transitorio,
   "interval_off": pp_tempo_ref_desligado,
   "limit_off": pp_valor_ref_desligado,
   "pre_cut": pp_pre_corte_transitorio,
   "variable_off": pp_var_ref_desligado
  })

for pro in pre_process:
    df_newdata ,_,_ = drop_transitorio_desligado(df_newdata,pro["variable_off"],pro["limit_off"],pro["interval_off"],timestamp,pre_corte=pro["pre_cut"],pos_corte=pro["after_cut"])
print(f"Dataset shape after remove offs: {df_newdata.shape}")
df_newdata.head()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_newdata['Timestamp'],
    y=df_newdata["762P0034.PV"],
    mode='lines',
    name='762P0034.PV',
    line=dict(color='black')
))

fig.update_layout(
    template='plotly_white',
    hovermode='x unified'
)
fig.show()

## Extract Features

### Extract Features with tsfresh

In [ ]:
df_predict = df_newdata.set_index(timestamp)

variables = df_predict.drop(columns=["Falhas"])
labels = df_predict["Falhas"]

# parâmetros de janela
window = pd.Timedelta("60min")
step = pd.Timedelta("30min")

# criar índices baseados em passo fixo
df_predict = df_predict.copy()
df_predict["window_start"] = df_predict.index.floor(step)
df_predict["window_id"] = ((df_predict.index - df_predict["window_start"]) // step).cumsum()

# corrigir para garantir tamanho de janela correto
valid_windows = df_predict.groupby("window_id").filter(
    lambda x: (x.index.max() - x.index.min()) >= window
)

# long format direto
long_format = (
    valid_windows
    .reset_index()
    .melt(id_vars=[timestamp, "window_id"], var_name="variable", value_name="value")
)

# extrair features
X_features = extract_features(
    long_format,
    column_id="window_id",
    column_sort=timestamp,
    column_kind="variable",
    column_value="value",
    n_jobs=1
)

X_features = impute(X_features)

# rótulo: 1 se houver falha dentro da janela
y_features = labels.groupby(df_predict["window_id"]).apply(lambda x: int(x.any()))
y_features = y_features.loc[X_features.index]

print("Shape final de X_features:", X_features.shape)
print("Shape final de y_features:", y_features.shape)

### Extract Features with functions

In [ ]:
## utils
def shannon_entropy(arr, bins=10):
    if arr.size == 0:
        return np.nan
    p, _ = np.histogram(arr, bins=bins, density=True)
    p = p[p > 0]
    if p.size == 0:
        return np.nan
    return -np.sum(p * np.log(p))

def safe_autocorr(s, lag=1):
    try:
        return s.autocorr(lag=lag)
    except Exception:
        return np.nan

# function to extract features from a window
def features_from_window(g):
    out = {}
    for col in g.columns:
        s = g[col].dropna()
        prefix = col
        if s.empty:
            for name in ["mean","std","var","min","max","median","skew","kurtosis",
                         "q25","q75","iqr","amplitude","energy","entropy",
                         "autocorr_lag1","autocorr_lag2","coeff_var"]:
                out[f"{prefix}_{name}"] = np.nan
            continue

        arr = s.values
        out[f"{prefix}_mean"] = s.mean()
        out[f"{prefix}_std"] = s.std()
        out[f"{prefix}_var"] = s.var()
        out[f"{prefix}_min"] = s.min()
        out[f"{prefix}_max"] = s.max()
        out[f"{prefix}_median"] = s.median()
        out[f"{prefix}_skew"] = s.skew()
        out[f"{prefix}_kurtosis"] = float(kurtosis(arr, fisher=True))
        out[f"{prefix}_q25"] = np.percentile(arr, 25)
        out[f"{prefix}_q75"] = np.percentile(arr, 75)
        out[f"{prefix}_iqr"] = out[f"{prefix}_q75"] - out[f"{prefix}_q25"]
        out[f"{prefix}_amplitude"] = out[f"{prefix}_max"] - out[f"{prefix}_min"]
        out[f"{prefix}_energy"] = np.sum(arr ** 2)
        out[f"{prefix}_entropy"] = shannon_entropy(arr, bins=10)
        out[f"{prefix}_autocorr_lag1"] = safe_autocorr(s, lag=1)
        out[f"{prefix}_autocorr_lag2"] = safe_autocorr(s, lag=2)
        mean_val = out[f"{prefix}_mean"]
        out[f"{prefix}_coeff_var"] = (out[f"{prefix}_std"] / mean_val) if mean_val not in (0, np.nan) else np.nan

    return pd.Series(out)


# params
window = "60min"
step = "30min"

df_predict = df_newdata.set_index(timestamp)
variables = df_predict.drop(columns=["Falhas"], errors='ignore')

# build sliding windows
starts = pd.date_range(df_predict.index.min(), df_predict.index.max() - pd.Timedelta(window), freq=step)

X_list, y_list, idx_list = [], [], []

for start in starts:
    end = start + pd.Timedelta(window)
    window_vars = variables.loc[start:end]
    window_labels = labels.loc[start:end]

    if window_vars.empty:
        continue

    # extract features
    feats = features_from_window(window_vars)
    X_list.append(feats)

    # index representing the center of the window
    idx_list.append(start + (pd.Timedelta(window) / 2))

# build final DataFrames
X_features = pd.DataFrame(X_list, index=idx_list)

# remove samples with any NaN in X_features
mask_valid = ~X_features.isna().any(axis=1)
X_features = X_features.loc[mask_valid]

print("Shape final de X_features:", X_features.shape)

### Basic Feature Extraction

In [ ]:
df_predict = df_newdata.set_index(timestamp)

variables = df_predict.drop(columns=["Falhas"])
labels = df_predict["Falhas"]

# windows parameters
window = pd.Timedelta("60min")
step = pd.Timedelta("30min")

# init point of each window
starts = pd.date_range(df_predict.index.min(), df_predict.index.max() - window, freq=step)

X_list, y_list, idx_list = [], [], []

for start in starts:
    end = start + window
    window_vars = variables.loc[start:end]
    window_labels = labels.loc[start:end]

    if window_vars.empty or window_labels.empty:
        continue

    # Extract basic features
    feats = window_vars.agg(["mean", "std", "min", "max", "median", "skew", "var"]).T
    feats.index = [f"{col}" for col in feats.index]  # keep variable names
    feats = feats.stack()  # "long"
    feats.index = [f"{var}_{stat}" for var, stat in feats.index]
    X_list.append(feats)

    # any failure in the window = label 1
    y_list.append(int(window_labels.any()))

    # index represent the center of the window
    idx_list.append(start + window / 2)

# Final DataFrames
X_features = pd.DataFrame(X_list, index=idx_list)
y_features = pd.Series(y_list, index=idx_list, name="Falhas")

# Clean NaN values
X_features = X_features.dropna()
y_features = y_features.loc[X_features.index]

print("Shape final de X_features:", X_features.shape)
print("Shape final de y_features:", y_features.shape)

## RandomForest Predict

In [ ]:
pred_proba_rf = rf_clf.predict_proba(X_features)[:, 1]

fig2 = go.Figure()
fig2.add_trace(go.Scatter(
    x=X_features.index,
    y=pred_proba_rf*100,
    mode='lines',
    name='RandomForest',
    line=dict(color='blue')
))
fig2.update_layout(
    template='plotly_white',
    hovermode='x unified',
    title='RandomForest',
    yaxis_title='Probabilidade de Falha (%)',
    xaxis_title='Tempo'
)
fig2.show()

## XGBoost Predict

In [ ]:
pred_proba_xgb = xgb_clf.predict_proba(X_features.to_numpy())[:, 1]

fig3 = go.Figure()
fig3.add_trace(go.Scatter(
    x=X_features.index,
    y=pred_proba_xgb*100,
    mode='lines',
    name='XGBoost',
    line=dict(color='green')
))
fig3.update_layout(
    template='plotly_white',
    hovermode='x unified',
    title='XGBoost',
    yaxis_title='Probabilidade de Falha (%)',
    xaxis_title='Tempo'
)
fig3.show()

# TimeSeries Classification sktime Algorithms

## ROCKET

### Format data to sktime

In [ ]:
def format_data_for_sktime(df, window_duration, overlap_duration, target_col='Falhas'):
    """
    format dataframe to sktime format for time series classification

    Args:
        df (pd.DataFrame): DataFrame with DatetimeIndex.
        window_duration (str): window duration.
        overlap_duration (str): overlap duration.
        target_col (str): target column name.

    Returns:
        tuple[pd.DataFrame, pd.Series]:
            - X: DataFrame in pd-multiindex format (instance, timestamp).
            - y: series with labels for each instance.
    """
    # convert durations to timedelta
    window_td = pd.to_timedelta(window_duration)
    overlap_td = pd.to_timedelta(overlap_duration)
    step_td = window_td - overlap_td # step between windows

    # List to store windows
    window_list = []
    # List to store labels for each window
    y_list = []

    start_time = df.index[0]
    end_time = df.index[-1]
    
    window_start = start_time
    instance_id = 0

    while window_start + window_td <= end_time:
        window_end = window_start + window_td
        
        # select data in the current window
        window_df = df.loc[window_start:window_end].iloc[:-1] # iloc[:-1] for exactly window size 60
        
        # garantee window has the correct size
        if window_df.shape[0] == window_td.total_seconds() / 60:
            # associate instance_id to the window
            window_df['instance_id'] = instance_id
            window_list.append(window_df)

            # define label for the window: 1 if any failure in the window, else 0
            # the .any() function is more robust than .mode() for rare events.
            label = 1 if window_df[target_col].any() else 0
            y_list.append(pd.Series(label, index=[instance_id]))
            
            instance_id += 1
        
        # move to next window
        window_start += step_td

    if not window_list:
        print("Nenhuma janela completa foi criada. Verifique as durações e o tamanho do DataFrame.")
        return None, None
        
    # concatenate all windows into a single DataFrame
    X_sktime = pd.concat(window_list)
    X_sktime = X_sktime.drop(columns=[target_col])
    X_sktime = X_sktime.set_index(['instance_id', X_sktime.index])
    X_sktime.index.names = ['instances', 'timepoints']

    # concatenate labels into a single Series
    y_sktime = pd.concat(y_list)
    y_sktime.index.name = 'instances'

    return X_sktime, y_sktime

## apply function
window = "60min"
overlap = "30min"

df_sktime = df_dataset.copy().set_index(timestamp)
X, y = format_data_for_sktime(df_sktime, window, overlap, target_col='Falhas')

## display results
if X is not None:
    print(f"format of X (features): {X.shape}")
    print(f"format of y (target): {y.shape}\n")

X

### Split train test data

In [ ]:
from sktime.split import temporal_train_test_split

y_train_sktime, y_test_sktime, X_train_sktime, X_test_sktime = temporal_train_test_split(
    y, X, test_size=0.3
)

### Apply ROCKET

In [ ]:
rocket = Rocket(num_kernels=600, random_state=42)
rocket.fit(X_train_sktime)

X_train_feat = rocket.transform(X_train_sktime)
X_test_feat = rocket.transform(X_test_sktime)

print("Shape features após Rocket:", X_train_feat.shape)

### SMOTE in training set

In [ ]:
sm = SMOTE(random_state=42)
X_train_sktime_balanced, y_train_sktime_balanced = sm.fit_resample(X_train_feat, y_train_sktime)

print("Distribution after SMOTE:", Counter(y_train_sktime_balanced))

### Fit model

In [ ]:
## RidgeClassifierCV is recommended by the ROCKET paper
rdgcv_clf = RidgeClassifierCV(alphas=np.logspace(-4, 4, 100))
rdgcv_clf.fit(X_train_sktime_balanced, y_train_sktime_balanced)

## Predict
y_pred_rocket = rdgcv_clf.predict(X_test_feat)

accuracy = balanced_accuracy_score(y_test_sktime, y_pred_rocket)
f1 = f1_score(y_test_sktime, y_pred_rocket, average='weighted')
recall = recall_score(y_test_sktime, y_pred_rocket, average='weighted')
precision = precision_score(y_test_sktime, y_pred_rocket, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")

## Plot ROC Curve
y_scores = rdgcv_clf.decision_function(X_test_feat)

lb = LabelBinarizer()
y_test_bin = lb.fit_transform(y_test_sktime)

if y_test_bin.shape[1] == 1:
    y_test_bin = y_test_bin.ravel()

fpr, tpr, thresholds = roc_curve(y_test_bin, y_scores)
roc_auc = roc_auc_score(y_test_bin, y_scores)

plt.figure(figsize=(6, 6))
plt.plot(fpr, tpr, color="blue", lw=2, label=f"ROC curve (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], color="gray", lw=1, linestyle="--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend(loc="lower right")
plt.grid(alpha=0.3)
plt.show()

### Plot predictions X real labels

In [ ]:
y_features_test.sort_index(inplace=True)

fig_pred = go.Figure()
fig_pred.add_trace(go.Scatter(
    y=y_pred_rocket,
    name='Predict',
    mode="lines",
    line=dict(color='black')
))
fig_pred.add_trace(go.Scatter(
    y=y_test_sktime,
    name='Real',
    mode="lines",
    line=dict(color='red')
))
fig_pred.show()

## WEALSEL + MUSE

In [ ]:
from sktime.classification.dictionary_based import MUSE

# Fit Model
muse_clf = MUSE(anova=True, variance=False, bigrams=True, window_inc=2, alphabet_size=4, use_first_order_differences=True, feature_selection='chi2', p_threshold=0.05, support_probabilities=False, n_jobs=1, random_state=42)
muse_clf.fit(X_train_sktime, y_train_sktime)

# Predict
y_pred_muse = muse_clf.predict(X_test_sktime)

accuracy = balanced_accuracy_score(y_test_sktime, y_pred_muse)
f1 = f1_score(y_test_sktime, y_pred_muse, average='weighted')
recall = recall_score(y_test_sktime, y_pred_muse, average='weighted')
precision = precision_score(y_test_sktime, y_pred_muse, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")

# DeepLearning

## Split Train-Test Data

In [ ]:
X = df_dataset.drop(columns=['Falhas', 'Timestamp'])
y = df_dataset['Falhas']

X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.3, 
    random_state=42, 
    shuffle=False
)

## Scalling training set
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)


## Verify class distribution in train and test sets
print("Class distribution in training set (%):")
print(y_train.value_counts(normalize=True) * 100)
print("\nClass distribution in test set (%):")
print(y_test.value_counts(normalize=True) * 100)

## Aplly SMOTE in train data

In [ ]:
def create_windows(X, y, window_size):
    X_windows, y_windows = [], []
    # Usamos .values para trabalhar com arrays numpy, que é mais rápido
    x_vals = X.values
    y_vals = y.values
    
    for i in range(len(X) - window_size + 1):
        X_windows.append(x_vals[i:i + window_size])
        # Rótulo da janela é 1 se houver qualquer falha nela
        y_windows.append(np.any(y_vals[i:i + window_size]))
        
    return np.array(X_windows), np.array(y_windows).astype(int)

window_size = 60
X_train_windows, y_train_windows = create_windows(X_train_scaled, y_train, window_size)
X_test_windows, y_test_windows = create_windows(X_test_scaled, y_test, window_size)

print(f"Formato das janelas de treino (X): {X_train_windows.shape}")
print(f"Formato dos rótulos de treino (y): {y_train_windows.shape}")
print(f"Distribuição de classes no treino (janelas): {Counter(y_train_windows)}")
print("-" * 30)


# 4. APLICAÇÃO DO SMOTE (Nas janelas de treino)
# O SMOTE precisa de dados 2D, então achatamos as janelas temporariamente
n_samples, n_timesteps, n_features = X_train_windows.shape
X_train_windows_flat = X_train_windows.reshape(n_samples, -1)

print(f"Aplicando SMOTE nas {n_samples} janelas de treino...")
sm = SMOTE(random_state=42)
X_train_balanced_flat, y_train_balanced = sm.fit_resample(X_train_windows_flat, y_train_windows)

# Remodelar os dados de volta para o formato 3D (amostras, timesteps, features)
X_train_balanced = X_train_balanced_flat.reshape(-1, n_timesteps, n_features)

print(f"\nFormato de X de treino após SMOTE: {X_train_balanced.shape}")
print(f"Distribuição de classes após SMOTE: {Counter(y_train_balanced)}")
print("-" * 30)

## 1D-CNN

In [ ]:
window_size = 60  # 60 time steps (minutes)
n_features = X.shape[1]  # number of features

# model parameters
input_shape = (window_size, n_features)
n_classes = 1

# 1D-CNN model
def create_cnn_model(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv1D(filters=64, kernel_size=3, activation='relu')(inputs)
    x = Conv1D(filters=64, kernel_size=3, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = GlobalMaxPooling1D()(x)
    outputs = Dense(n_classes, activation='sigmoid')(x)
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

## InceptionTime

In [ ]:
# InceptionTime model
def create_inceptiontime_model(input_shape, n_filters=32, kernel_sizes=[3, 5, 11]):
    inputs = Input(shape=input_shape)
    
    # Inception module
    conv_list = []
    for ks in kernel_sizes:
        conv = Conv1D(filters=n_filters, kernel_size=ks, padding='same', activation='relu')(inputs)
        conv_list.append(conv)
    
    x = Concatenate()(conv_list)
    x = GlobalMaxPooling1D()(x)
    x = Dropout(0.5)(x)
    outputs = Dense(n_classes, activation='sigmoid')(x)
    
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model